# Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

## Assignment Part 1: Create dataframe to match

### Import all the required libraries

Pandas, urlopen and BeautifulSoup to be used

In [71]:
# import pandas library for data analsysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# import library to handle requests
import requests

# tranform JSON file into a pandas dataframe
from pandas import json_normalize

# import numpy
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import urlopen to access internet data
from urllib.request import urlopen

# import Beautiful soup to parse html page
from bs4 import BeautifulSoup

#import folium for generating map
import folium

# import k-means from clustering stage
from sklearn.cluster import KMeans

# confirm that all libraries have been imported
print('Libraries imported!')

Libraries imported!


### Obtain data from specified url

Will use urlopen and read() to get the html codes of the link. The html codes will then be parsed through BeautifulSoup to allow me to access the specific table.

In [72]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# use urlopen to open url and read() to obtain html codes
html = urlopen(url).read()
# use BeautifulSoup to parse html
raw_data = BeautifulSoup(html,'html.parser')

### Obtain column header from web table using the html tags 'th'

The table column names are found to be wrapped within the html tags 'th'. So, I'll go ahead and get it and feed it into a column_header list.

In [98]:
column_tags = raw_data('th')
# create empty list for column header
column_header = []
# iterate across tags, get the text, strip \n at the end and append to column header list
for tags in column_tags:
    header = tags.text.rstrip()
    column_header.append(header)

### Initialize dataframe using the column names

The dataframe is then initialised using the column_header list, dropping 'Canadian postal codes' as it is not required as part of the assignment.

In [99]:
df = pd.DataFrame(columns = column_header).drop(columns='Canadian postal codes')
df

,Postal Code,Borough,Neighborhood


### Obtain row data from each row using the html tags 'tr'

Each row of information is found between the html tags 'tr'

In [100]:
row_tags = raw_data('tr')
# iterate across each row tags except 0 as this is the column header row and the last 4 rows which is not relevant
for tags in row_tags[1:len(row_tags)-4]:
        row_postal_code = tags('td')[0].text.rstrip()
        row_borough = tags('td')[1].text.rstrip()
        row_neighborhood = tags('td')[2].text.rstrip()
        #ignore row where borough is not assigned
        if row_borough != 'Not assigned':
            #append each borough assigned row into dataframe
            df = df.append({'Postal Code':row_postal_code,
                            'Borough':row_borough,
                            'Neighborhood':row_neighborhood},ignore_index=True)
df.head(15)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Explore how many boroughs and postal code within each borough

In [101]:
print('There are {} boroughs, {} neighborhood and {} postal codes in Toronto.'.
      format(len(df['Borough'].unique()),len(df['Neighborhood'].unique()),len(df['Postal Code'].unique())))

There are 10 boroughs, 98 neighborhood and 103 postal codes in Toronto.


Seems like there are duplicates of neighborhood in a couple of postal codes in Toronto. Let's find out what they are

In [102]:
df.groupby(['Borough','Neighborhood']).count().sort_values(by=['Postal Code'],ascending=False).head()

Postal Code
Borough         Neighborhood                                         
North York      Downsview                                           4
                Don Mills                                           2
                Willowdale                                          2
Central Toronto Davisville                                          1
North York      North Park, Maple Leaf Park, Upwood Park            1

Seems like Donsview, Don Mills and Willowdale has multiple postal codes. To make it distinct we can append the postal codes at the end of the neighborhoods

In [103]:
df['Neighborhood'] = df[['Neighborhood','Postal Code']].apply(lambda x: '-'.join(x),axis=1)

Let's check again

In [104]:
df.groupby(['Borough','Neighborhood']).count().sort_values(by=['Postal Code'],ascending=False).head()

Postal Code
Borough         Neighborhood                                     
Central Toronto Davisville North-M4P                            1
North York      Lawrence Manor, Lawrence Heights-M6A            1
Scarborough     Agincourt-M1S                                   1
North York      York Mills, Silver Hills-M2L                    1
                York Mills West-M2P                             1

Now I have distinct neighborhoods, it will allow me to map out venues in distinctive postal code areas. Another solution is to remove rows of duplicate neighborhoods but I decided to keep it going forwards. 

In [105]:
print(df.shape)

(103, 3)


## Assignment Part II: Combining previous dataframe with geospatial data

I am going to use the geosptial_data.csv instead of Geocoder due to inability to obtain data from the library

### Download csv file and read geospatial data

In [11]:
!wget -q -O Geospatial_data.csv https://cocl.us/Geospatial_data    

### Read geospatial.csv file

In [106]:
geo_spatial = pd.read_csv('Geospatial_data.csv')
geo_spatial.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Merge geo_spatial data with main df

In [107]:
df = pd.merge(df,geo_spatial,how='left',on=['Postal Code'])

In [108]:
df2 = df
df2['Neighborhood'] = df2['Neighborhood'].str[:-4]
print(df.shape)
df2.head(15)

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Assignment Part III: Exploration and clustering of neighborhoods in Toronto

### Use geopy to obtain longitude and latitude of Toronto city for use in folium

In [84]:
from geopy.geocoders import Nominatim

Define instance for geocoder and name agent at 'Toronto_explorer'

In [85]:
address = 'Toronto City, Canada'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


Create map of Toronto City with neighborhoods superimposed on top. The neighborhoods corresponds to each postal code and note that neighborhoods Donsview, Don Mills and Willowdale has multiple postal codes.

In [86]:
# create map of Toronto City using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

I am going to explore all the neighborhoods and segment them by using the Foursquare API

### Explore one of Toronto's neighborhood

Let's explore one of the neighbourhood(s) in Toronto. We will use the first one. Let's get the latitude and longitude for that.

In [20]:
neighborhood_latitude = df.loc[0,'Latitude']
neighborhood_longitude = df.loc[0,'Longitude']
neighborhood_name = df.loc[0,'Neighborhood']

print('The {} neighborhood(s) has the latitude of {} and longitude of {}'.format(neighborhood_name,
                                                                                neighborhood_latitude,
                                                                                neighborhood_longitude))

The Parkwoods-M3A neighborhood(s) has the latitude of 43.7532586 and longitude of -79.3296565


Let's get the top 100 venues in that area with a 1000 metre radius. Create a GET url and obtain json

In [52]:
radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()

Borrowing the get_category_type function from earlier lab.

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean and restructure json data into a dataframme

In [25]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Brookbanks Park,Park,43.751976,-79.332140
2,Tim Hortons,Café,43.760668,-79.326368
3,A&W,Fast Food Restaurant,43.760643,-79.326865
4,Bruno's valu-mart,Grocery Store,43.746143,-79.324630


Now I have the venues for one neighborhood. How many venues were returned by Foursquare?

In [26]:
print('{} venues were returned by Foursquare for the {} neighborhood.'.format(nearby_venues.shape[0],
                                                                             neighborhood_name))

29 venues were returned by Foursquare for the Parkwoods-M3A neighborhood.


### Explore all of Toronto's neighborhood

Borrowing function from earlier labs to repeat the above steps for all neighborhoods in Toronto

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
                   
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Use above defined function and run it for all neighborhoods in Toronto and put it to *toronto_venues*

In [28]:
print(df.shape)
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


(103, 5)
Parkwoods-M3A
Victoria Village-M4A
Regent Park, Harbourfront-M5A
Lawrence Manor, Lawrence Heights-M6A
Queen's Park, Ontario Provincial Government-M7A
Islington Avenue-M9A
Malvern, Rouge-M1B
Don Mills-M3B
Parkview Hill, Woodbine Gardens-M4B
Garden District, Ryerson-M5B
Glencairn-M6B
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale-M9B
Rouge Hill, Port Union, Highland Creek-M1C
Don Mills-M3C
Woodbine Heights-M4C
St. James Town-M5C
Humewood-Cedarvale-M6C
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood-M9C
Guildwood, Morningside, West Hill-M1E
The Beaches-M4E
Berczy Park-M5E
Caledonia-Fairbanks-M6E
Woburn-M1G
Leaside-M4G
Central Bay Street-M5G
Christie-M6G
Cedarbrae-M1H
Hillcrest Village-M2H
Bathurst Manor, Wilson Heights, Downsview North-M3H
Thorncliffe Park-M4H
Richmond, Adelaide, King-M5H
Dufferin, Dovercourt Village-M6H
Scarborough Village-M1J
Fairview, Henry Farm, Oriole-M2J
Northwood Park, York University-M3J
East Toronto-M4J
Harbourfront 

Checking the size of the resulting dataframe

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(4911, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods-M3A,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods-M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods-M3A,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods-M3A,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods-M3A,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


Group the venues by neighborhood to check how many venues are found per neighborhood

In [30]:
print(toronto_venues.groupby('Neighborhood').count().shape)
toronto_venues.groupby('Neighborhood').count()

(102, 6)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt-M1S,44,44,44,44,44,44
"Alderwood, Long Branch-M8W",27,27,27,27,27,27
"Bathurst Manor, Wilson Heights, Downsview North-M3H",30,30,30,30,30,30
Bayview Village-M2K,14,14,14,14,14,14
"Bedford Park, Lawrence Manor East-M5M",41,41,41,41,41,41
Berczy Park-M5E,100,100,100,100,100,100
"Birch Cliff, Cliffside West-M1N",14,14,14,14,14,14
"Brockton, Parkdale Village, Exhibition Place-M6K",100,100,100,100,100,100
Business reply mail Processing Centre-M7Y,49,49,49,49,49,49


Seems like there is a neighborhood which has not return venues at all. Which one is it?

In [31]:
venues_grouped = toronto_venues.groupby('Neighborhood').count().reset_index()
missing_neigh=df[~df.Neighborhood.isin(venues_grouped.Neighborhood)]
missing_neigh

,Postal Code,Borough,Neighborhood,Latitude,Longitude
95,M1X,Scarborough,Upper Rouge-M1X,43.836125,-79.205636


Let's confirm by trying a Foursquare API call

In [32]:
missing_latitude = missing_neigh.loc[95,'Latitude']
missing_longitude = missing_neigh.loc[95,'Longitude']
radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, missing_latitude, missing_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec19e2b9388d7001c2709d8'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Rouge',
  'headerFullLocation': 'Rouge, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.84512470900002,
    'lng': -79.19318232223782},
   'sw': {'lat': 43.827124690999995, 'lng': -79.21808987776217}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

There was indeed no results. We'll have to ensure that this row of Postal Code M1X and Neighborhood Upper Rouge is not reflected later.

Investigate how many unique categories there are from all returned values

In [33]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 327 uniques categories.


Analyzing each neighborhood through one hot encoding

In [63]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot)
fixed_columns.insert(0,fixed_columns.pop(fixed_columns.index('Neighborhood')))
toronto_onehot = toronto_onehot[fixed_columns]

# Clean up neighborhood column as venues data has been retrieved
toronto_onehot['Neighborhood'] = toronto_onehot['Neighborhood'].str[:-4]
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,Souvlaki Shop,Spa,

What is the shape of the new toronto dataframe?

In [64]:
toronto_onehot.shape

(4911, 327)

The shape matches the total number of venue categories and number of venues found. Now let's group them by the neighborhood to get an idea of the frequency of venue occurences in each category

In [65]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,Souvlaki Shop,Spa,

Let's check the shape to make sure it is valid

In [66]:
toronto_grouped.shape

(97, 327)

The shape corresponds to the main df shape of 103 minus the neighborhood Upper Rouge which did not return any result from Foursquare APO

Borrowing function to rate the top 10 location for each neighborhood

In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now to create a dataframe of neighborhoods with the top 10 most common venue categories

In [87]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
# match new dataframe 'Neighborhood' column with the toronto_grouped dataframe
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(97, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Bakery,Caribbean Restaurant,Motorcycle Shop,Bank,Coffee Shop,Sushi Restaurant,Supermarket,Latin American Restaurant
1,"Alderwood, Long Branch",Discount Store,Pharmacy,Park,Convenience Store,Pizza Place,Coffee Shop,Pub,Sandwich Place,Trail,Shopping Mall
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Park,Coffee Shop,Bank,Pharmacy,Community Center,Fried Chicken Joint,Sushi Restaurant,Supermarket,Mediterranean Restaurant
3,Bayview Village,Japanese Restaurant,Grocery Store,Gas Station,Bank,Chinese Restaurant,Restaurant,Shopping Mall,Café,Park,Trail
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Bank,Park,Sandwich Place,Fast Food Restaurant,Bagel Shop,Bakery,Juice Bar,Sushi Restaurant


### Now time to cluster these neighborhoods

Run k-means to cluster the neighborhood into 6 clusters.

In [88]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 3, 5, 5, 2, 2, 3, 2, 2, 2])

Now lets combine the cluster labels with the top 10 venues

In [89]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# create a merge dataframe but drop
toronto_merged = df2.drop(index=95)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Park,Shopping Mall,Convenience Store,Pharmacy,Bus Stop,Fish & Chips Shop,Supermarket,Food & Drink Shop,Fast Food Restaurant,Skating Rink
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Coffee Shop,Gym / Fitness Center,Hockey Arena,Portuguese Restaurant,Lounge,Park,Playground,Pizza Place,Golf Course,Men's Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Theater,Café,Restaurant,Park,Diner,Italian Restaurant,Pub,Bakery,Breakfast Spot
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Vietnamese Restaurant,Dessert Shop,Fried Chicken Joint,Sushi Restaurant,Furniture / Home Store,Event Space
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Park,Thai Restaurant,Gastropub,Ramen Restaurant,Café,Italian Restaurant,Ice Cream Shop


Now let's visualise the clusters

In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the clusters

__Cluster 1__

In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1,2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,"Parkview Hill, Woodbine Gardens",Brewery,Pizza Place,Fast Food Restaurant,Gym / Fitness Center,Construction & Landscaping,Athletics & Sports,Soccer Stadium,Breakfast Spot,Café,Gastropub
18,Scarborough,"Guildwood, Morningside, West Hill",Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Greek Restaurant,Bus Line,Sports Bar,Breakfast Spot,Smoothie Shop,Supermarket
34,North York,"Northwood Park, York University",Coffee Shop,Pizza Place,Furniture / Home Store,Restaurant,Sports Bar,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,Fast Food Restaurant,Frame Store
38,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",Discount Store,Chinese Restaurant,Coffee Shop,Pizza Place,Fast Food Restaurant,Grocery Store,Sandwich Place,Intersection,Light Rail Station,Bank
51,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",Pizza Place,Ice Cream Shop,Beach,Sports Bar,Burger Joint,Park,Cajun / Creole Restaurant,Hardware Store,Farmers Market,Electronics Store
64,York,Weston,Pizza Place,Train Station,Coffee Shop,Fried Chicken Joint,Pharmacy,Skating Rink,Sandwich Place,Diner,Furniture / Home Store,Grocery Store
70,Etobicoke,Westmount,Pizza Place,Gas Station,Breakfast Spot,Intersection,Middle Eastern Restaurant,Supermarket,Chinese Restaurant,Golf Course,Golf Driving Range,Sandwich Place
77,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",Pharmacy,Bank,Intersection,Bus Line,Supermarket,Supplement Shop,Shopping Mall,Beer Store,Gas Station,Chinese Restaurant
89,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",Grocery Store,Pizza Place,Pharmacy,Gym Pool,Caribbean Restaurant,Park,Sandwich Place,Beer Store,Fried Chicken Joint,Liquor Store


Cluster 1 seems to cover areas which generally has pizza place and fast food restautant. Looking at the map, it seems to be mostly outside Toronto City centre hence the assumption is that cluster 1 is mostly __Food and Beverage__ areas.

__Cluster 2__

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1,2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,"Rouge Hill, Port Union, Highland Creek",Breakfast Spot,Burger Joint,Park,Playground,Italian Restaurant,Fish Market,Farm,Eastern European Restaurant,Flower Shop,Electronics Store
101,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Park,Italian Restaurant,Shopping Mall,Ice Cream Shop,Eastern European Restaurant,Gym / Fitness Center,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant


Cluster 2 covers smaller area of park and smaller food joints. Cluster 2 can be assumed to be __Special Community__ areas.

__Cluster 3__

In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1,2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,Victoria Village,Coffee Shop,Gym / Fitness Center,Hockey Arena,Portuguese Restaurant,Lounge,Park,Playground,Pizza Place,Golf Course,Men's Store
2,Downtown Toronto,"Regent Park, Harbourfront",Coffee Shop,Theater,Café,Restaurant,Park,Diner,Italian Restaurant,Pub,Bakery,Breakfast Spot
3,North York,"Lawrence Manor, Lawrence Heights",Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Vietnamese Restaurant,Dessert Shop,Fried Chicken Joint,Sushi Restaurant,Furniture / Home Store,Event Space
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Japanese Restaurant,Park,Thai Restaurant,Gastropub,Ramen Restaurant,Café,Italian Restaurant,Ice Cream Shop
7,North York,Don Mills,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Bank,Supermarket,Burger Joint,Asian Restaurant,Mobile Phone Shop,Beer Store
9,Downtown Toronto,"Garden District, Ryerson",Coffee Shop,Gastropub,Japanese Restaurant,Theater,Café,Restaurant,Hotel,Italian Restaurant,Diner,Middle Eastern Restaurant
13,North York,Don Mills,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Bank,Supermarket,Burger Joint,Asian Restaurant,Mobile Phone Shop,Beer Store
15,Downtown Toronto,St. James Town,Coffee Shop,Café,Restaurant,Gastropub,Seafood Restaurant,Theater,Italian Restaurant,Cocktail Bar,Cosmetics Shop,Plaza
16,York,Humewood-Cedarvale,Convenience Store,Pizza Place,Coffee Shop,Hockey Arena,Trail,Field,Middle Eastern Restaurant,Mexican Restaurant,Café,Frozen Yogurt Shop
19,East Toronto,The Beaches,Coffee Shop,Pub,Pizza Place,Breakfast Spot,Japanese Restaurant,Beach,Bakery,Sandwich Place,Caribbean Restaurant,Bar


Cluster 3 covers areas of greatest mixture of coffee shop, cafe, restaurant, hotels and bars. Again from the map, these are closer to the Toronto city centre. Hence the assumption is that cluster 3 is __Downtown and Business__ areas.

__Cluster 4__

In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1,2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,Park,Shopping Mall,Convenience Store,Pharmacy,Bus Stop,Fish & Chips Shop,Supermarket,Food & Drink Shop,Fast Food Restaurant,Skating Rink
5,Etobicoke,Islington Avenue,Pharmacy,Convenience Store,Bakery,Shopping Mall,Golf Course,Park,Skating Rink,Grocery Store,Café,Playground
11,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",Park,Pizza Place,Hotel,Restaurant,Clothing Store,Grocery Store,Gym,Bank,Fish & Chips Shop,Mexican Restaurant
14,East York,Woodbine Heights,Park,Coffee Shop,Sandwich Place,Pizza Place,Thai Restaurant,Curling Ice,Café,Skating Rink,Cosmetics Shop,Liquor Store
21,York,Caledonia-Fairbanks,Pharmacy,Bus Stop,Park,Pizza Place,Mexican Restaurant,Bank,Coffee Shop,Grocery Store,Discount Store,Japanese Restaurant
22,Scarborough,Woburn,Park,Coffee Shop,Chinese Restaurant,Pharmacy,Mobile Phone Shop,Indian Restaurant,Fast Food Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
27,North York,Hillcrest Village,Coffee Shop,Park,Pharmacy,Bakery,Korean Restaurant,Shopping Mall,Bank,Chinese Restaurant,Grocery Store,Restaurant
50,North York,Humber Summit,Electronics Store,Arts & Crafts Store,Pharmacy,Bakery,Shopping Mall,Park,Italian Restaurant,Bank,Pizza Place,Field
58,Scarborough,"Birch Cliff, Cliffside West",Park,Gym Pool,Café,General Entertainment,Convenience Store,Thai Restaurant,Ice Cream Shop,Restaurant,College Stadium,Diner
88,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",Park,Bakery,Indian Restaurant,Liquor Store,Skating Rink,Fried Chicken Joint,Dessert Shop,Italian Restaurant,Grocery Store,Pharmacy


Cluster 4 covers areas of parks, malls, stores, pharmacy and retails store. Looking at the map the are outside of the Toronto city centre. Cluster 4 can be assumed to be __Retail Parks__ area.

__Cluster 5__

In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1,2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,"York Mills, Silver Hills",Park,Pool,Zoo,Farmers Market,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


Cluster 5 covers areas of park, pool and zoo. Cluster 5 can be assumed to be __Recreational__.

__Cluster 6__

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1,2] + list(range(6, toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,"Malvern, Rouge",Restaurant,Trail,Coffee Shop,Fast Food Restaurant,Spa,Chinese Restaurant,Bank,Bakery,Caribbean Restaurant,Paper / Office Supplies Store
10,North York,Glencairn,Grocery Store,Fast Food Restaurant,Gym,Pizza Place,Gas Station,Coffee Shop,Tennis Court,Japanese Restaurant,Fish Market,Metro Station
17,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Coffee Shop,Pet Store,Beer Store,Café,Grocery Store,Transportation Service,Liquor Store,College Rec Center,Intersection,Fish & Chips Shop
23,East York,Leaside,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Grocery Store,Electronics Store,Sports Bar,Bank,Brewery,Burger Joint,Restaurant
26,Scarborough,Cedarbrae,Coffee Shop,Bank,Gas Station,Indian Restaurant,Bakery,Intersection,Burger Joint,Fast Food Restaurant,Sporting Goods Shop,Music Store
28,North York,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Park,Coffee Shop,Bank,Pharmacy,Community Center,Fried Chicken Joint,Sushi Restaurant,Supermarket,Mediterranean Restaurant
29,East York,Thorncliffe Park,Coffee Shop,Indian Restaurant,Grocery Store,Burger Joint,Brewery,Turkish Restaurant,Gym,Sandwich Place,Supermarket,Bank
32,Scarborough,Scarborough Village,Ice Cream Shop,Convenience Store,Restaurant,Bowling Alley,Fast Food Restaurant,Grocery Store,Coffee Shop,Train Station,Japanese Restaurant,Sandwich Place
39,North York,Bayview Village,Japanese Restaurant,Grocery Store,Gas Station,Bank,Chinese Restaurant,Restaurant,Shopping Mall,Café,Park,Trail
40,North York,Downsview,Coffee Shop,Vietnamese Restaurant,Grocery Store,Hotel,Pizza Place,Park,Gas Station,Sandwich Place,Fast Food Restaurant,Turkish Restaurant


Cluster 6 covers areas of greatest restaurants, grocery stores and parks. Looking at the map, the cluster are generally outisde  the Toronto City centre. Hence, the assumption is that this cluster is mainly __General Residential__ areas